In [1]:
import pandas  as pd
from langchain.llms import OpenAI
from dotenv import load_dotenv

import openai,os
load_dotenv(r'D:\Git\NLP\LLM\ActivLoop\.env')
openai_api_key = os.getenv("ACTIVELOOP_TOKEN")

assert openai_api_key, "ERROR: Azure OpenAI Key is missing"
openai.api_key = openai_api_key

openai.api_base = os.getenv("OpenAiService")
openai.api_type = "azure"
openai.api_version =os.getenv("OpenAiVersion")
davincimodel= os.getenv("OpenAiDavinci")
active_loop_token=os.getenv("ACTIVELOOP_TOKEN")
embedding_model=os.getenv("OpenAiEmbedding")
chat_ai=os.getenv("ChatAI")#
HUGGINGFACEHUB_API_TOKEN=os.getenv("HUGGINGFACEHUB_API_TOKEN")

c:\Users\sri.karan\.conda\envs\activeloop\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (3.6.17) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In this lesson we will explore how LangChain, Deep Lake, and GPT-4 can be used to develop a sales assistant able to give advice to salesman, taking into considerations internal guidelines.

### Introduction
This particular article provides an in-depth view of a sales call assistant that connects you to a chatbot that understands the context of your conversation. A great feature of SalesCopilot is its ability to detect potential objections from customers and deliver recommendations on how best to address them.

The article is a journey that reveals the challenges faced and the solutions discovered during the creation of a the project. You'll learn about the two distinct text-splitting approaches that didn't work and how these failures paved the way for an effective solution.

Firstly, the authors tried to rely solely on the LLM, but they encountered issues such as response inconsistency and slow response times with GPT-4. Secondly, they naively split the custom knowledge base into chunks, but this approach led to context misalignment and inefficient results.

After these unsuccessful attempts, a more intelligent way of splitting the knowledge base based on its structure was adopted. This change greatly improved the quality of responses and ensured better context grounding for LLM responses. This process is explained in detail, helping you grasp how to navigate similar challenges in your own AI projects.

Next, the article explores how SalesCopilot was integrated with Deep Lake. This integration enhanced SalesCopilot's capabilities by retrieving the most relevant responses from a custom knowledge base, thereby creating a persistent, efficient, and highly adaptable solution for handling customer objections.

By the end of this lesson, you'll learn how to utilize LLMs, how to intelligently split your knowledge base, and integrate it with a vector database like Deep Lake for optimal performance.

# https://www.activeloop.ai/resources/conversation-intelligence-gong-io-open-source-alternative-ai-sales-assistant/

In [2]:
text = """
Objection: "There's no money."
It could be that your prospect's business simply isn't big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.

Objection: "We don't have any budget left this year."
A variation of the "no money" objection, what your prospect's telling you here is that they're having cash flow issues. But if there's a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.

Objection: "We need to use that budget somewhere else."
Prospects sometimes try to earmark resources for other uses. It's your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a massive ROI with you.
"""

# Split the text into a list using the keyword "Objection: "
objections_list = text.split("Objection: ")[1:]  # We ignore the first split as it is empty

# Now, prepend "Objection: " to each item as splitting removed it
objections_list = ["Objection: " + objection for objection in objections_list]


In [3]:
objections_list

['Objection: "There\'s no money."\nIt could be that your prospect\'s business simply isn\'t big enough or generating enough cash right now to afford a product like yours. Track their growth and see how you can help your prospect get to a place where your offering would fit into their business.\n\n',
 'Objection: "We don\'t have any budget left this year."\nA variation of the "no money" objection, what your prospect\'s telling you here is that they\'re having cash flow issues. But if there\'s a pressing problem, it needs to get solved eventually. Either help your prospect secure a budget from executives to buy now or arrange a follow-up call for when they expect funding to return.\n\n',
 'Objection: "We need to use that budget somewhere else."\nProspects sometimes try to earmark resources for other uses. It\'s your job to make your product/service a priority that deserves budget allocation now. Share case studies of similar companies that have saved money, increased efficiency, or had a

In [4]:
import os
import re
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DeepLake

class DeepLakeLoader:
    def __init__(self, source_data_path):
        self.source_data_path = source_data_path
        self.file_name = os.path.basename(source_data_path) # What we'll name our database 
        self.data = self.split_data()
        if self.check_if_db_exists():
            self.db = self.load_db()
        else:
            self.db = self.create_db()
    def split_data(self):  
        """  
        Preprocess the data by splitting it into passages.  

        If using a different data source, this function will need to be modified.  

        Returns:  
            split_data (list): List of passages.  
        """  
        with open(self.source_data_path, 'r') as f:  
            content = f.read()  
        split_data = re.split(r'(?=\d+\. )', content)
        if split_data[0] == '':  
            split_data.pop(0)  
        split_data = [entry for entry in split_data if len(entry) >= 30]  
        return split_data
    def check_if_db_exists(self):
        """
        Check if the database already exists.

        Returns:
            bool: True if the database exists, False otherwise.
        """
        return os.path.exists(f'deeplake/{self.file_name}')

    def load_db(self):  
        """  
        Load the database if it already exists.  

        Returns:  
            DeepLake: DeepLake object.  
        """
        embeddings = OpenAIEmbeddings(deployment=embedding_model)
        my_activeloop_org_id="hayagriva99999"
        my_activeloop_dataset_name = "sales_coplot"
        dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
        return DeepLake(dataset_path=dataset_path, embedding_function=embeddings,read_only=True)  
        #return DeepLake(dataset_path=f'deeplake/{self.file_name}', embedding_function=OpenAIEmbeddings(deployemnt=embedding_model), read_only=True)  

    def create_db(self):  
        """  
        Create the database if it does not already exist.  

        Databases are stored in the deeplake directory.  

        Returns:  
            DeepLake: DeepLake object.  
        """  
        embeddings = OpenAIEmbeddings(deployment=embedding_model)
        my_activeloop_org_id="hayagriva99999"
        my_activeloop_dataset_name = "sales_coplot"
        dataset_path = f"hub://{my_activeloop_org_id}/{my_activeloop_dataset_name}"
        db=DeepLake.from_texts(self.data[:16],embeddings,dataset_path=dataset_path)
        # db.add_documents(self.data)
        return db

        # return DeepLake.from_texts(self.data, embeddings, dataset_path=dataset_path)

    def query_db(self, query):  
        """  
        Query the database for passages that are similar to the query.  

        Args:  
            query (str): Query string.  

        Returns:  
            content (list): List of passages that are similar to the query.  
        """  
        results = self.db.similarity_search(query, k=3)  
        content = []  
        for result in results:  
            content.append(result.page_content)  
        return content



In [5]:
content = "1. First item. 2. Second item. 3. Third item."
re.split(r'(?=\d+\. )', content)

['', '1. First item. ', '2. Second item. ', '3. Third item.']

In [6]:
db = DeepLakeLoader(r'D:\Git\NLP\LLM\ActivLoop\data\SalesTesting.txt')


Deep Lake Dataset in hub://hayagriva99999/sales_coplot already exists, loading from the storage


|

Dataset(path='hub://hayagriva99999/sales_coplot', tensors=['embedding', 'id', 'metadata', 'text'])

  tensor      htype      shape      dtype  compression
  -------    -------    -------    -------  ------- 
 embedding  embedding  (32, 1536)  float32   None   
    id        text      (32, 1)      str     None   
 metadata     json      (32, 1)      str     None   
   text       text      (32, 1)      str     None   


In [7]:
LIVE_CHAT_PROMPT = """
Reminder: You're SalesCopilot.
Your goal is to help the user in their sales call with the customer. 
Using conversation transcripts, you'll help create responses and guide the user (labeled You).
Keep your responses helpful, concise, and relevant to the conversation.  
The transcripts may be fragmented, incomplete, or even incorrect. Do not ask for clarification, do your best to understand what
the transcripts say based on context. Be sure of everything you say.
Keep responses concise and to the point. Starting now, answer the user's question based on the transcript:

"""

DETECT_OBJECTION_PROMPT = """
Your task is to read the transcript and discern whether the customer is raising any objections to the product or service the salesperson is selling.
If the customer is simply stating their thoughts, preferences, or facts that are not specifically connected to the product or service, it is not an objection. 
Quote only from the transcript.
Do not add, infer, or interpret anything.
Example:
'''
Customer: I'm not sure if I can afford this. It's a bit expensive. The sky is blue. I like the color blue. 

You: I'm not sure if I can afford this. It's a bit expensive.
'''
If there is no objection, respond with 'None'.
Starting now, you will respond only with either the quote or None: 
"""

OBJECTION_GUIDELINES_PROMPT = """
You are SalesCopilot. You will be provided with a customer objection, and a selection
of guidelines on how to respond to certain objections. 
Using the provided content, write out the objection and the actionable course of action you recommend.
Objections sound like:
'''It's too expensive.
There's no money.
We don't have any budget left.
I need to use this budget somewhere else.
I don't want to get stuck in a contract.
We're already working with another vendor.
I'm locked into a contract with a competitor.
I can get a cheaper version somewhere else.'''
 
Example of your message:

'It seems like the customer is {explain their objection}.

I recommend you {course of action for salesperson}.'

"""

SAVED_TRANSCRIPT_PROMPT ="""
You are SalesCopilot. You will be provided with a transcript of a sales call between the user and a customer.
Answer any questions the user asks you. You may also assess the user's performance and provide feedback.
The transcripts may be fragmented, incomplete, or even incorrect. Do not ask for clarification, do your best to understand what
the transcripts say based on context.
The speaker labeled "You" in the transcripts is the user you are helping.
"""

In [8]:
results = db.query_db("It's too expensive.")


In [9]:
results

['1. "It\'s too expensive."\nCommon sales objections and rebuttals about price\n\nPrice objections are the most common type of objection and are even voiced by prospects who have every intention of buying. Beware — the moment you start focusing on price as a selling point, you reduce yourself to a transactional middleman. Instead, circle back to the product\'s value.\n\nExample Rebuttal\n"I\'d love to unpack [product\'s] features and how it can help with the issue of [prospect problem] you shared with me."\n\n',
 '1. "It\'s too expensive."\nCommon sales objections and rebuttals about price\n\nPrice objections are the most common type of objection and are even voiced by prospects who have every intention of buying. Beware — the moment you start focusing on price as a selling point, you reduce yourself to a transactional middleman. Instead, circle back to the product\'s value.\n\nExample Rebuttal\n"I\'d love to unpack [product\'s] features and how it can help with the issue of [prospect 

In [10]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage

chat = ChatOpenAI(engine=chat_ai)
system_message = SystemMessage(content=OBJECTION_GUIDELINES_PROMPT)
human_message = HumanMessage(content=f'Customer objection: {"Its too expensive."} | Relevant guidelines: {results}')

response = chat([system_message, human_message])

WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


In [11]:
results

['1. "It\'s too expensive."\nCommon sales objections and rebuttals about price\n\nPrice objections are the most common type of objection and are even voiced by prospects who have every intention of buying. Beware — the moment you start focusing on price as a selling point, you reduce yourself to a transactional middleman. Instead, circle back to the product\'s value.\n\nExample Rebuttal\n"I\'d love to unpack [product\'s] features and how it can help with the issue of [prospect problem] you shared with me."\n\n',
 '1. "It\'s too expensive."\nCommon sales objections and rebuttals about price\n\nPrice objections are the most common type of objection and are even voiced by prospects who have every intention of buying. Beware — the moment you start focusing on price as a selling point, you reduce yourself to a transactional middleman. Instead, circle back to the product\'s value.\n\nExample Rebuttal\n"I\'d love to unpack [product\'s] features and how it can help with the issue of [prospect 

In [12]:
response.content

'It seems like the customer is objecting on the basis of the price being too high.\n\nI recommend you to focus on the value of the product and how it can help solve the customer\'s problem. Avoid making price the main selling point as it may reduce you to a transactional middleman. Instead, offer to unpack the product\'s features and explain how it can assist with the customer\'s issue. For instance, you could say, "I\'d love to unpack our product\'s features and how it can help with the issue you shared with me."'